In [1]:
import pandas as pd
import openai
openai.api_key = ""
df = pd.read_pickle("./data/all_posts_US_EN.pkl")
#df = pd.read_pickle("../ct_df_posts_single_preds.pkl")

In [2]:
df=df[df["country"]=="US"]
df=df[df["language"]=="en"]
df = df[df["caption"]!=""]
#df.to_pickle("data/all_posts_US_EN.pkl")

In [2]:
df2 = df.groupby("username").count().sort_values(ascending=False,by="caption").iloc[:-10]
author_sample = list(df2.sample(20,random_state=1).index)
del df2

In [3]:
df = df[df["username"].isin(author_sample)]
df = df.groupby("username").apply(lambda x: x.sample(n=50,random_state=1, replace=True))

In [ ]:
# takes 1 minute per author for 50 posts
df2 = pd.DataFrame (["author"]+["text"]+[99]*1536).transpose()
for username in df.index.get_level_values("username").unique():
    i=0
    for caption in df.loc[username]["caption"]:
        embedding = openai.Embedding.create(input = [caption],
                    model="text-embedding-ada-002")['data'][0]['embedding']
        df2.loc[len(df2)] = [f"{username}{i}"]+[caption]+embedding
        i+=1
    # just to ensure that it isn't stuck
    if len(df2)%100==1:print(f"User {username} done")

In [9]:
import pickle
with open("5k-embeddings.pkl","wb") as f:
    pickle.dump(df2,f)

In [26]:
import pickle
with open("data/5k-embeddings.pkl","rb") as f:
    df = pickle.load(f)

In [37]:
from umap import UMAP
from bertopic import BERTopic
reducer = UMAP(n_neighbors=15, min_dist=0.1, n_components=75, random_state=42)
umap_embeddings = reducer.fit_transform(df.iloc[1:4001,2:].values)

In [38]:
# takes 20 seconds, but nonsense?
topic_model = BERTopic(min_topic_size=50)
#umap_embeddings = topic_model._reduce_dimensionality(df2.iloc[1:4001,2:].values)

In [42]:
umap_embeddings.shape
topic_model.fit(df.iloc[1:4001,1], embeddings=umap_embeddings)
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,381,-1_this_the_to_of
1,0,579,0_by_the_de_with
2,1,462,1_the_to_this_of
3,2,368,2_and_to_the_you
4,3,235,3_in_the_beach_this
5,4,233,4_the_and_birthday_you
6,5,198,5_the_of_to_for
7,6,192,6_legend_prince_man_booty
8,7,190,7_my_the_you_outfit
9,8,137,8_pepe_photo_with_lospicantes


In [34]:
topic_model = BERTopic(min_topic_size=50)
topic_model.fit_transform(df[1])
topic_model.get_topic_info()

,Topic,Count,Name
0,0,3905,0_the_to_and_in
1,1,102,1_orca_koko_keegs_karl


In [25]:
import numpy as np
topic_model.fit(df2.iloc[1:4001,1], embeddings=np.array(df2.iloc[1:4001,2:]))
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1606,-1_the_with_to_and
1,0,188,0_yoga_practice_to_and
2,1,187,1_yea_need_yup_me
3,2,106,2_devourpower_tag_nyc_devourpowertv
4,3,99,3_youaretheentireuniverse_race_2019_heaven
...,...,...,...
65,64,11,64_gbfmovie_misterwillett_tribeca_xoshroq
66,65,10,65_09_urban_teamnewfriends_preschool
67,66,10,66_qls_questlovesupreme_about_music
68,67,10,67_dirty_bathroom_room_dirtysouth


In [28]:
topic_model = BERTopic(min_topic_size=10)
topics, probs = topic_model.fit_transform(df[1].iloc[1:].reset_index(drop=True))

In [ ]:
# use the same sample with less topics
# concatenate the posts to find categories of influencers
# consider simple clustering with GPT